Mindfactory GPU Crawler
by Ralf Neureither
V 1.1

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import re

In [2]:
product_info_dict={"name":[],"no_sold":[],"price":[]} #Grundlage des späteren Pandas DF

rtx_urls=["https://www.mindfactory.de/Hardware/Grafikkarten+(VGA)/GeForce+RTX+fuer+Gaming/RTX+2080+Ti.html","https://www.mindfactory.de/Hardware/Grafikkarten+(VGA)/GeForce+RTX+fuer+Gaming/RTX+2080.html","https://www.mindfactory.de/Hardware/Grafikkarten+(VGA)/GeForce+RTX+fuer+Gaming/RTX+2080.html/page/2","https://www.mindfactory.de/Hardware/Grafikkarten+(VGA)/GeForce+RTX+fuer+Gaming/RTX+2070.html","https://www.mindfactory.de/Hardware/Grafikkarten+(VGA)/GeForce+RTX+fuer+Gaming/RTX+2070.html/page/2","https://www.mindfactory.de/Hardware/Grafikkarten+(VGA)/GeForce+RTX+fuer+Gaming/RTX+2060.html","https://www.mindfactory.de/Hardware/Grafikkarten+(VGA)/GeForce+RTX+fuer+Gaming/RTX+2060.html/page/2","https://www.mindfactory.de/Hardware/Grafikkarten+(VGA)/GeForce+RTX+fuer+Gaming/RTX+2060.html/page/3"]
gtx_urls=["https://www.mindfactory.de/Hardware/Grafikkarten+(VGA)/GeForce+GTX+fuer+Gaming/GTX+1070.html","https://www.mindfactory.de/Hardware/Grafikkarten+(VGA)/GeForce+GTX+fuer+Gaming/GTX+1660+Ti.html","https://www.mindfactory.de/Hardware/Grafikkarten+(VGA)/GeForce+GTX+fuer+Gaming/GTX+1660.html","https://www.mindfactory.de/Hardware/Grafikkarten+(VGA)/GeForce+GTX+fuer+Gaming/GTX+1650.html","https://www.mindfactory.de/Hardware/Grafikkarten+(VGA)/GeForce+GTX+fuer+Gaming/GTX+1060.html","https://www.mindfactory.de/Hardware/Grafikkarten+(VGA)/GeForce+GTX+fuer+Gaming/GTX+1050+Ti.html","https://www.mindfactory.de/Hardware/Grafikkarten+(VGA)/GeForce+GTX+fuer+Gaming/GTX+1050.html"]
polaris_urls=["https://www.mindfactory.de/Hardware/Grafikkarten+(VGA)/Radeon+RX+Serie/RX+590.html","https://www.mindfactory.de/Hardware/Grafikkarten+(VGA)/Radeon+RX+Serie/RX+580.html","https://www.mindfactory.de/Hardware/Grafikkarten+(VGA)/Radeon+RX+Serie/RX+570.html","https://www.mindfactory.de/Hardware/Grafikkarten+(VGA)/Radeon+RX+Serie/RX+560.html","https://www.mindfactory.de/Hardware/Grafikkarten+(VGA)/Radeon+RX+Serie/RX+550.html"]
rdna_urls=["https://www.mindfactory.de/Hardware/Grafikkarten+(VGA)/Radeon+RX+Serie/RX+5700+XT.html","https://www.mindfactory.de/Hardware/Grafikkarten+(VGA)/Radeon+RX+Serie/RX+5700.html",]
vega_url="https://www.mindfactory.de/Hardware/Grafikkarten+(VGA)/Radeon+RX+VEGA.html"
r7_url="https://www.mindfactory.de/Hardware/Grafikkarten+(VGA)/Radeon+VII.html"

gpu_cat_urls=[i for i in rtx_urls]+[i for i in gtx_urls]+[i for i in polaris_urls]+[i for i in rdna_urls]
gpu_cat_urls.append(vega_url)
gpu_cat_urls.append(r7_url)
gpu_cat_urls

['https://www.mindfactory.de/Hardware/Grafikkarten+(VGA)/GeForce+RTX+fuer+Gaming/RTX+2080+Ti.html',
 'https://www.mindfactory.de/Hardware/Grafikkarten+(VGA)/GeForce+RTX+fuer+Gaming/RTX+2080.html',
 'https://www.mindfactory.de/Hardware/Grafikkarten+(VGA)/GeForce+RTX+fuer+Gaming/RTX+2080.html/page/2',
 'https://www.mindfactory.de/Hardware/Grafikkarten+(VGA)/GeForce+RTX+fuer+Gaming/RTX+2070.html',
 'https://www.mindfactory.de/Hardware/Grafikkarten+(VGA)/GeForce+RTX+fuer+Gaming/RTX+2070.html/page/2',
 'https://www.mindfactory.de/Hardware/Grafikkarten+(VGA)/GeForce+RTX+fuer+Gaming/RTX+2060.html',
 'https://www.mindfactory.de/Hardware/Grafikkarten+(VGA)/GeForce+RTX+fuer+Gaming/RTX+2060.html/page/2',
 'https://www.mindfactory.de/Hardware/Grafikkarten+(VGA)/GeForce+RTX+fuer+Gaming/RTX+2060.html/page/3',
 'https://www.mindfactory.de/Hardware/Grafikkarten+(VGA)/GeForce+GTX+fuer+Gaming/GTX+1070.html',
 'https://www.mindfactory.de/Hardware/Grafikkarten+(VGA)/GeForce+GTX+fuer+Gaming/GTX+1660+Ti.htm

In [3]:
for url in gpu_cat_urls:
    http=requests.get(url)
    site=bs(http.content,"html.parser")
    div_list=site.find_all("div",class_="pcontent")
    
    for element in div_list:
        p_link_list=[]
        p_link_list.append(element.find_all("a")[0].get("href"))
        p_link_list=list(dict.fromkeys(p_link_list))
        for product in p_link_list:
            product_site=requests.get(product)
            product_site_soup=bs(product_site.content,"html.parser")
            product_info_dict["name"].append(product_site_soup.title.string.split("|")[0].strip())
            price_match=product_site_soup.find_all("div",class_="pprice")[0].find_all("span",class_="text-currency")[0].next_sibling.string
            price=re.search(r"(\d.*\d+,*\d+)",price_match)[0]
            product_info_dict["price"].append(price)
            try:
                zero_sold_test=product_site_soup.find_all("div", class_="row mat10 vbottom")[0].find_all("div", class_="psold")[0]
                if "über" in str(zero_sold_test):
                    product_info_dict["no_sold"].append(product_site_soup.find_all(class_="pcountsold")[0].string)
                else:
                    product_info_dict["no_sold"].append("0")
            except:
                product_info_dict["no_sold"].append("0")
            

In [4]:
gpus_df=pd.DataFrame(product_info_dict)
gpus_df.shape

(534, 3)

In [5]:
gpus_df.drop_duplicates(inplace=True)

In [6]:
gpus_df.head()

,name,no_sold,price
0,11GB MSI GeForce RTX 2080 Ti VENTUS GP (Retail...,170,"1.088,95"
1,11GB Palit GeForce RTX 2080 Ti Dual Aktiv PCIe...,460,"1.078,85"
2,11GB Gigabyte GeForce RTX 2080 Ti Gaming OC 11...,230,"1.236,85"
3,11GB Gigabyte GeForce RTX 2080 Ti Windforce Ak...,90,"1.152,45"
4,11GB KFA2 GeForce RTX 2080 Ti Hall Of Fame Akt...,20,1.549


Splitten wäre im nachfolgenden Absatz wohl sinnvoller gewesen.

In [7]:
gpus_df.dtypes

name       object
no_sold    object
price      object
dtype: object

In [8]:
gpus_df.head()

,name,no_sold,price
0,11GB MSI GeForce RTX 2080 Ti VENTUS GP (Retail...,170,"1.088,95"
1,11GB Palit GeForce RTX 2080 Ti Dual Aktiv PCIe...,460,"1.078,85"
2,11GB Gigabyte GeForce RTX 2080 Ti Gaming OC 11...,230,"1.236,85"
3,11GB Gigabyte GeForce RTX 2080 Ti Windforce Ak...,90,"1.152,45"
4,11GB KFA2 GeForce RTX 2080 Ti Hall Of Fame Akt...,20,1.549


In [9]:
def clean_and_enrich_df(gpus_df):
    
    
    if gpus_df.dtypes["price"]=="object":
        gpus_df["price"]=gpus_df["price"].str.replace(".","")

        gpus_df["price"]=gpus_df["price"].str.replace(",",".")

        gpus_df["price"]=pd.to_numeric(gpus_df["price"])

    if gpus_df.dtypes["no_sold"]=="object":
        gpus_df["no_sold"]=gpus_df["no_sold"].str.replace(".","")
        
        gpus_df["no_sold"]=pd.to_numeric(gpus_df["no_sold"])

    gpus_df.loc[gpus_df["name"].str.contains("2060 SUPER"),"gpu_model"]="2060 SUPER"

    gpus_df.loc[gpus_df["name"].str.contains("2070 SUPER"),"gpu_model"]="2070 SUPER"

    gpus_df.loc[(gpus_df["name"].str.contains("2060"))&(gpus_df["gpu_model"]!="2060 SUPER"),"gpu_model"]="2060"

    gpus_df.loc[(gpus_df["name"].str.contains("2070"))&(gpus_df["gpu_model"]!="2070 SUPER"),"gpu_model"]="2070"

    gpus_df.loc[gpus_df["name"].str.contains("2080 Ti"),"gpu_model"]="2080 Ti"

    gpus_df.loc[(gpus_df["name"].str.contains("2080"))&(gpus_df["gpu_model"]!="2080 Ti"),"gpu_model"]="2080"

    gpus_df.loc[gpus_df["gpu_model"].str.contains("2060|2070|2080",na=False),"arch"]="Turing"

    gpus_df.loc[gpus_df["name"].str.contains("5700"),"arch"]="RDNA"

    gpus_df.loc[gpus_df["name"].str.contains("5700 XT"),"gpu_model"]="5700 XT"

    gpus_df.loc[(gpus_df["name"].str.contains("5700"))&(gpus_df["gpu_model"].isnull()),"gpu_model"]="5700 Non-XT"

    gpus_df.loc[gpus_df["name"].str.contains("Vega"),"arch"]="Vega"

    gpus_df.loc[gpus_df["name"].str.contains("Vega 56"),"gpu_model"]="Vega56"

    gpus_df.loc[gpus_df["name"].str.contains("Vega 64"),"gpu_model"]="Vega64"

    gpus_df.loc[gpus_df["name"].str.contains("1050|1060|1070|1080",na=False),"arch"]="Pascal"

    gpus_df.loc[gpus_df["name"].str.contains("1650|1660",na=False),"arch"]="Turing"

    gpus_df.loc[gpus_df["name"].str.contains("1050 Ti"),"gpu_model"]="1050 Ti"
    gpus_df.loc[(gpus_df["name"].str.contains("1050"))&(gpus_df["gpu_model"].isnull()),"gpu_model"]="1050"

    gpus_df.loc[gpus_df["name"].str.contains("1060"),"gpu_model"]="1060"

    gpus_df.loc[gpus_df["name"].str.contains("1070 Ti"),"gpu_model"]="1070 Ti"
    gpus_df.loc[(gpus_df["name"].str.contains("1070"))&(gpus_df["gpu_model"].isnull()),"gpu_model"]="1070"

    gpus_df.loc[gpus_df["name"].str.contains("1080 Ti"),"gpu_model"]="1080 Ti"
    gpus_df.loc[(gpus_df["name"].str.contains("1080"))&(gpus_df["gpu_model"].isnull()),"gpu_model"]="1080"

    gpus_df.loc[gpus_df["name"].str.contains("1660 Ti"),"gpu_model"]="1660 Ti"
    gpus_df.loc[(gpus_df["name"].str.contains("1660"))&(gpus_df["gpu_model"].isnull()),"gpu_model"]="1660"

    gpus_df.loc[gpus_df["name"].str.contains("1650"),"gpu_model"]="1650"

    gpus_df.loc[gpus_df["name"].str.contains("590"),"gpu_model"]="590"

    gpus_df.loc[gpus_df["name"].str.contains("580"),"gpu_model"]="580"

    gpus_df.loc[(gpus_df["name"].str.contains("570"))&(gpus_df["gpu_model"].isnull()),"gpu_model"]="570"

    gpus_df.loc[gpus_df["name"].str.contains("560"),"gpu_model"]="560"

    gpus_df.loc[gpus_df["name"].str.contains("550"),"gpu_model"]="550"

    gpus_df.loc[gpus_df["name"].str.contains("VII"),"gpu_model"]="RadeonVII"

    gpus_df.loc[gpus_df["name"].str.contains("VII"),"arch"]="Vega"

    gpus_df.loc[(gpus_df["name"].str.contains("590|580|570|560|550"))&(gpus_df["arch"].isnull()),"arch"]="Polaris"

    gpus_df.loc[gpus_df["arch"].str.contains("Turing|Pascal",na=False),"manufacturer"]="Nvidia"

    gpus_df.loc[gpus_df["arch"].str.contains("Polaris|Vega|RDNA",na=False),"manufacturer"]="AMD"

    gpus_df["vram_size"]=gpus_df["name"].str.split(expand=True)[0]
    
    return gpus_df

In [10]:
gpus_df=clean_and_enrich_df(gpus_df)

In [11]:
gpus_df.isnull().sum()

name            0
no_sold         0
price           0
gpu_model       0
arch            0
manufacturer    0
vram_size       0
dtype: int64

In [12]:
from datetime import datetime

In [13]:
gpus_df["timestamp"]=datetime.now()

In [14]:
gpus_df.head()

,name,no_sold,price,gpu_model,arch,manufacturer,vram_size,timestamp
0,11GB MSI GeForce RTX 2080 Ti VENTUS GP (Retail...,170,1088.95,2080 Ti,Turing,Nvidia,11GB,2019-12-01 19:42:02.221760
1,11GB Palit GeForce RTX 2080 Ti Dual Aktiv PCIe...,460,1078.85,2080 Ti,Turing,Nvidia,11GB,2019-12-01 19:42:02.221760
2,11GB Gigabyte GeForce RTX 2080 Ti Gaming OC 11...,230,1236.85,2080 Ti,Turing,Nvidia,11GB,2019-12-01 19:42:02.221760
3,11GB Gigabyte GeForce RTX 2080 Ti Windforce Ak...,90,1152.45,2080 Ti,Turing,Nvidia,11GB,2019-12-01 19:42:02.221760
4,11GB KFA2 GeForce RTX 2080 Ti Hall Of Fame Akt...,20,1549.00,2080 Ti,Turing,Nvidia,11GB,2019-12-01 19:42:02.221760


In [15]:
t_stamp_string="_"+str(datetime.today().year)+"_"+str(datetime.today().month)+"_"+str(datetime.today().day)+"_"+str(datetime.today().hour)

In [16]:
gpus_df.to_csv(r"C:\Users\Acedia23\Documents\Python\Own Projects\gpus_df{}.csv".format(t_stamp_string))